<a href="https://colab.research.google.com/github/GovindRegar/Property-Valuation-System/blob/main/Property_Valuation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kora -q 
from kora.selenium import wd
import pandas as pd
from selenium.webdriver.common.by import By


In [ ]:
# url="https://www.99acres.com/3-bhk-bedroom-apartment-flat-for-sale-in-hinjewadi-phase-2-pune-1070-sq-ft-spid-L53049788"
def link_data(link_url):
  wd.get(link_url)
  try:
    specification=wd.find_element_by_class_name("banner__sizePropType").text[0:5]
  except:
    try:
      specification=wd.find_element_by_id("bedWash").text[:5]
    except:
      specification=None

  # project=wd.find_element_by_class_name("banner__projectName").text
  try:
    floors=wd.find_element_by_id("floorCountSociety").text
  except:
    try:
      floors=wd.find_element_by_id("floorNumLabel").text[0]
    except:
      floors=None
  try:
    status=wd.find_element_by_class_name("factTableComponent__npPossession").text
  except:
    try:
      status=wd.find_element_by_id("Availability_Lbl").text
    except:
      status=None

  try:
    poss=wd.find_element_by_xpath("/html/body/div[1]/div/div/div/div[3]/div/div[2]/div[1]/div[3]/div/div[2]/span[2]").text
  except:
    try:
        poss=wd.find_element_by_id("agePossessionLbl").text
    except:
      poss=None

    # area=wd.find_element_by_xpath("/html/body/div[1]/div/div/div/div[3]/div/div[2]/div[1]/div[2]/div[1]/span").text

  # price=wd.find_element_by_xpath("/html/body/div[1]/div/div/div/div[3]/div/div[2]/div[1]/div[1]/div[2]/span[3]").text

  try:
    basic_amenities=wd.find_element_by_id("xidAmeWrap").text.split('\n')
  except:
    try:
      basic_amenities=wd.find_element_by_id("features").text.split('\n')
    except:
      basic_amenities=None

  try:
    lifestyle=wd.find_element_by_id("lifeStyleWrp").text.split('\n')
  except:
    lifestyle=None

  try:
    transaction_type=wd.find_element_by_id("Transact_Type_Label").text
  except:
    transaction_type="New sale"

  item={
      'specification':specification,
      # 'project':project,
      'floors':floors,
      'status':status,
      'possession':poss,
      # 'area':area,
      # 'price':price,
      'basic_amenities':basic_amenities,
      'lifestyle':lifestyle,
      'transaction_type':transaction_type
  }
  return item
# print(link_data(url))

In [ ]:
base_url="https://www.99acres.com/property-in-hinjewadi-pune-ffid-page-"

def dataset(url,n):
  all_items=[]#store all rows as a list of dict
  posted_by=[]
  
  for i in range(1,n):

    wd.get(url+str(i))
    tables=wd.find_element_by_class_name("srpTuple__tupleTable")
    dealers=wd.find_elements_by_class_name("srpTuple__srpDealerInfo")

    for d in dealers:
      name=d.find_element_by_class_name("list_header_semiBold").text
      posted_by.append(name)
   
    for table in tables:
      # try:
      #   Specification=table.find_element_by_xpath('.//*[@id="srp_tuple_property_title"]/h2').text
      # except:
      #   Specification=None
      try:
        Project=table.find_element_by_xpath('.//*[@id="srp_tuple_society_heading"]/a').text
      except:
        Project=None
      try:
        Area=table.find_element_by_xpath('.//*[@id="srp_tuple_primary_area"]').text
      except:
        Area=None
      try:
        sqft=table.find_element_by_xpath('.//*[@id="srp_tuple_primary_area_unit"]').text
      except:
        sqft=None
      try:
        sqmt=table.find_element_by_xpath('.//*[@id="srp_tuple_secondary_area"]').text
      except:
        sqmt=None
      try:
        Area=Area.replace(sqft,'')
        Area=Area.replace(sqmt,'')
        Area=Area.replace('\n','')
        Area=Area.replace(',','')
      except:
        None
      try:
        Area=Area[:Area.index('-')]
        Area=float(Area)
      except:
        None
      try:
        #price per sqft
        Price=table.find_element_by_xpath('.//*[@id="srp_tuple_price_per_unit_area"]').text
        Price=Price.replace('/sq.ft.','')
        Price=Price.replace(',','')
        Price=Price.replace('₹','')
        Price=float(Price)
      except:
        Price=None
      try:
        Link=table.find_element_by_tag_name('a').get_attribute('href')
      except:
        Link=None
      print(Link)
      # other_fields=link_data(Link)
      
  
      item={
          # 'specification':other_fields['specification'],
          # 'floors':other_fields['floors'],
          # 'status':other_fields['status'],
          # 'transaction_type':other_fields['transaction_type'],
          # 'basic_amenities':other_fields['basic_amenities'],
          # 'lifestyle':other_fields['lifestyle'],
          'project':Project,
          'area':Area,
          'price':Price,
          'link':Link
          }
      all_items.append(item)

  df1=pd.DataFrame(all_items)
  other_fields=[]
  for i in range(len(df1['link'])):
    print(i)
    item=link_data(df1['link'][i])
    other_fields.append(item)

  df2=pd.DataFrame(posted_by,columns=['posted_by'])
  df3=pd.DataFrame(other_fields)
  complete_df=pd.concat([df1,df2,df3],axis=1)
  return complete_df

data=dataset(base_url,2)
data.head()

# other_fields=[]
# for i in range(len(data['link'])):
#   item=link_data(data['link'][i])
#   other_fields.append(item)
# df_items=pd.DataFrame(all_items)
# df_items.head()
